In [11]:
# ==============================================================
# 1. Instalar las dependencias
# ==============================================================
!pip install -q sentence-transformers pandas pyarrow datasets gcsfs

In [12]:
# ==============================================================
# 2. Importar librerías
# ==============================================================

import pandas as pd
from sentence_transformers import SentenceTransformer


In [13]:
# ==============================================================
# 3. Cargar el modelo de embeddings (512D)
# ==============================================================

model = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")

# Verificar el funcionamiento del modelo ingresando una cadena de texto
vec = model.encode("Esta es una prueba para búsquedas semánticas")
print("Dimensión del embedding:", len(vec))  # Debe ser 512

Dimensión del embedding: 512


In [14]:
# ==============================================================
# 4. Cargar dataset de ejemplo
# ==============================================================

# Leemos el dataset
df = pd.read_csv("/content/drive/MyDrive/wine-reviews-1K.csv")

print("Dataset cargado desde CSV:")
print(df.head())

Dataset cargado desde CSV:
   num country                                        description  \
0    0      US  This tremendous 100% varietal wine hails from ...   
1    1   Spain  Ripe aromas of fig, blackberry and cassis are ...   
2    2      US  Mac Watson honors the memory of a wine once ma...   
3    3      US  This spent 20 months in 30% new French oak, an...   
4    4  France  This is the top wine from La Bégude, named aft...   

                            designation  points  price        province  \
0                     Martha's Vineyard      96  235.0      California   
1  Carodorum Selección Especial Reserva      96  110.0  Northern Spain   
2         Special Selected Late Harvest      96   90.0      California   
3                               Reserve      96   65.0          Oregon   
4                            La Brûlade      95   66.0        Provence   

             region1            region2             variety  \
0        Napa Valley               Napa  Cabernet 

In [15]:
# ==============================================================
# 5. Generar embeddings de una columna específica
# ==============================================================

columna_a_vectorizar = "description"

# Generar embeddings (lista de vectores de 512 floats)
embeddings = model.encode(df[columna_a_vectorizar].tolist())

# Añadir al DataFrame
df["embedding"] = embeddings.tolist()

print("\nDataset con embeddings (512D):")
print(df.head())



Dataset con embeddings (512D):
   num country                                        description  \
0    0      US  This tremendous 100% varietal wine hails from ...   
1    1   Spain  Ripe aromas of fig, blackberry and cassis are ...   
2    2      US  Mac Watson honors the memory of a wine once ma...   
3    3      US  This spent 20 months in 30% new French oak, an...   
4    4  France  This is the top wine from La Bégude, named aft...   

                            designation  points  price        province  \
0                     Martha's Vineyard      96  235.0      California   
1  Carodorum Selección Especial Reserva      96  110.0  Northern Spain   
2         Special Selected Late Harvest      96   90.0      California   
3                               Reserve      96   65.0          Oregon   
4                            La Brûlade      95   66.0        Provence   

             region1            region2             variety  \
0        Napa Valley               Napa  Cabe

In [16]:
# ==============================================================
# 6. Guardar resultados
# ==============================================================

# Guardar como CSV (embeddings en formato lista)
df.to_csv("dataset_con_embeddings.csv", index=False)

# Guardar como Parquet (más eficiente para vectores)
df.to_parquet("dataset_con_embeddings.parquet", index=False)

print("\nArchivos guardados: dataset_con_embeddings.csv / dataset_con_embeddings.parquet")




Archivos guardados: dataset_con_embeddings.csv / dataset_con_embeddings.parquet


In [17]:
# ==============================================================
# 7. Generar archivo SQL con UN SOLO INSERT masivo para PostgreSQL
# ==============================================================

import pandas as pd
import json
import math
import ast

# --------------------------------------------------------------
# Cargar el dataset con embeddings generados
# --------------------------------------------------------------
df = pd.read_csv("dataset_con_embeddings.csv")

# Si el campo 'embedding' está almacenado como string de lista, convertirlo
if isinstance(df["embedding"].iloc[0], str):
    df["embedding"] = df["embedding"].apply(ast.literal_eval)

# --------------------------------------------------------------
# Funciones auxiliares
# --------------------------------------------------------------
def to_sql_str(val):
    """Devuelve un valor escapado o NULL si falta."""
    if pd.isna(val) or val is None:
        return "NULL"
    s = str(val).replace("'", "''")
    return f"'{s}'"

def to_sql_num(val):
    """Devuelve un número o NULL si no es válido."""
    if val is None or (isinstance(val, float) and math.isnan(val)):
        return "NULL"
    return str(val)

# --------------------------------------------------------------
# Construir todas las tuplas con sus respectivos valores
# --------------------------------------------------------------
values = []
for _, row in df.iterrows():
    uid        = "NULL" if pd.isna(row.get("num")) else int(row["num"])
    country    = to_sql_str(row.get("country"))
    description= to_sql_str(row.get("description"))
    designation= to_sql_str(row.get("designation"))
    points     = to_sql_num(row.get("points"))
    price      = to_sql_num(row.get("price"))
    province   = to_sql_str(row.get("province"))
    region     = to_sql_str(row.get("region1") or row.get("region2"))
    variety    = to_sql_str(row.get("variety"))

    # Vector en formato ARRAY PostgreSQL (no JSON)
    # Ejemplo: '[0.12,0.34,0.56]'::vector
    vector_str = "[" + ",".join(map(str, row["embedding"])) + "]"
    vector_expr = f"'{vector_str}'::vector"

    values.append(f"({uid},{country},{description},{designation},{points},{price},{province},{region},{variety},{vector_expr})")

# --------------------------------------------------------------
# Crear INSERT masivo mediante un único INSERT
# --------------------------------------------------------------
insert_stmt = (
    "INSERT INTO winereviews "
    "(uid, country, description, designation, points, price, province, region, variety, description_vector)\n"
    "VALUES\n" + ",\n".join(values) + ";"
)

# --------------------------------------------------------------
# Guardar el resultado del INSERT en archivo .sql
# --------------------------------------------------------------
with open("insert_winereviews_massive_pg.sql", "w", encoding="utf-8") as f:
    f.write(insert_stmt)

print(f"Archivo 'insert_winereviews_massive_pg.sql' generado con {len(values)} filas.")


Archivo 'insert_winereviews_massive_pg.sql' generado con 1000 filas.


In [18]:
# ==============================================================
# 8. Ejemplos de consultas semánticas (PostgreSQL + pgvector)
# ==============================================================

import pandas as pd
import json
from sentence_transformers import SentenceTransformer

# --------------------------------------------------------------
# Ejemplos de consultas en lenguaje natural
# --------------------------------------------------------------
consultas = [
    "Vino elegante con aroma a cereza y taninos suaves",
    "Vino blanco afrutado con notas florales",
    "Vino tinto con cuerpo intenso y sabor a roble",
    "Aromas frescos y toques de vainilla",
    "Excelente vino de Napa Valley con alto puntaje",
    "Vino con sabores a mora, café, moca y roble",
    "Vino con notas a manzana, limón, cítricos y naranja"
]

# --------------------------------------------------------------
# Generar embeddings de cada consulta
# --------------------------------------------------------------
model = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")
embeddings_queries = model.encode(consultas, show_progress_bar=True)

# Guardar en DataFrame para revisión
df_queries = pd.DataFrame({
    "consulta": consultas,
    "embedding": embeddings_queries.tolist()
})

print("Consultas procesadas:")
display(df_queries.head())

# --------------------------------------------------------------
# Guardar embeddings de las consultas en CSV (opcional)
# --------------------------------------------------------------
df_queries.to_csv("consultas_con_embeddings.csv", index=False)
print("Archivo 'consultas_con_embeddings.csv' generado.")

# --------------------------------------------------------------
# Generar archivo SQL con las consultas semánticas
# --------------------------------------------------------------
sql_lines = []

for i, row in df_queries.iterrows():
    # Convertir el embedding en formato vector PostgreSQL: '[0.12,0.34,0.56]'
    query_vec_str = "[" + ",".join(map(str, row["embedding"])) + "]"
    consulta_texto = row["consulta"].replace("'", "''")  # Escapar comillas simples

    sql_lines.append(f"""
-- =====================================================
-- 🔍 Consulta semántica #{i+1}
-- Texto original: {consulta_texto}
-- =====================================================
-- Cosine distance (cuanto menor, más similar)
SELECT
    uid,
    country,
    province,
    variety,
    points,
    description,
    description_vector <=> '{query_vec_str}'::vector AS cosine_distance
FROM winereviews
ORDER BY cosine_distance ASC
LIMIT 5;

-- Euclidean distance
SELECT
    uid,
    country,
    province,
    variety,
    points,
    description,
    description_vector <-> '{query_vec_str}'::vector AS euclidean_distance
FROM winereviews
ORDER BY euclidean_distance ASC
LIMIT 5;

-- Inner product (si deseas usar vector_ip_ops)
SELECT
    uid,
    country,
    province,
    variety,
    points,
    description,
    description_vector <#> '{query_vec_str}'::vector AS inner_product
FROM winereviews
ORDER BY inner_product ASC
LIMIT 5;

""")

# --------------------------------------------------------------
# Guardar todas las consultas en un archivo .sql
# --------------------------------------------------------------
with open("consultas_embeddings_pg.sql", "w", encoding="utf-8") as f:
    f.write("\n".join(sql_lines))

print("Archivo 'consultas_embeddings_pg.sql' generado con las consultas semánticas.")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Consultas procesadas:


,consulta,embedding
0,Vino elegante con aroma a cereza y taninos suaves,"[-0.0420515201985836, -0.02215450629591942, 0...."
1,Vino blanco afrutado con notas florales,"[0.03517202287912369, 0.018681185320019722, -0..."
2,Vino tinto con cuerpo intenso y sabor a roble,"[-0.017544042319059372, 0.002918001264333725, ..."
3,Aromas frescos y toques de vainilla,"[0.02900838293135166, -0.0111842704936862, 0.0..."
4,Excelente vino de Napa Valley con alto puntaje,"[-0.0044096969068050385, -0.00591837614774704,..."


Archivo 'consultas_con_embeddings.csv' generado.
Archivo 'consultas_embeddings_pg.sql' generado con las consultas semánticas.


In [ ]:
!pip freeze > requirements_final.txt